# The CNN with ResNet backbone by transfer learning 5-10 K-fold Cross Validation

CHEST X-RAY IMAGES CLASSIFICATION WITH CNN

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import os, shutil, pathlib


print("tensorflow version:", tf.__version__)
print("numpy version:", np.__version__)
print("pandas version:", pd.__version__)

tensorflow version: 2.10.0
numpy version: 1.23.4
pandas version: 1.5.1


## Paths of Data

In [3]:
DATA_PATH = "../data"
RAW_DATASET_NAME = "20210708_görüntüler"   # raw dataset folder name
SUB_FOLDERS_PATH = f"{DATA_PATH}/{RAW_DATASET_NAME}"  # sub class folders

SUB_FOLDERS_PATH

'../data/20210708_görüntüler'

In [4]:
os.listdir(DATA_PATH)

['20210708_görüntüler', 'images_1', 'manuel_differet_person']

In [5]:
labels = {"covid+ ac grafileri" : "covid",
         "infiltratif akciğer hastası grafileri" : "infiltiratif",
         "normal akciğer grafileri": "normal"}

labels

{'covid+ ac grafileri': 'covid',
 'infiltratif akciğer hastası grafileri': 'infiltiratif',
 'normal akciğer grafileri': 'normal'}

# Prepare Data

In [6]:
def dataframe_for_folders(DATA_PATH, DATASET_NAME, labels):
    """create dataframe from data folders that every folder has one class data
    
    It prepares dataframe to put in  TF flow_from_dataframe function
    
    Args:
        DATA_PATH : parent folder relative path of DATASET_NAME. It contains all of data files.
        DATASET_NAME : dataset name which wanted to use - path from DATA_PATH to classes folders
        labels: a dict that contains folder names according to classes
        
    Returns:
        A pandas dataframe that has relative path to the dataset_name of data and labels
    """
    CLASS_FOLDERS_PATHS = os.listdir(f"{DATA_PATH}/{DATASET_NAME}")
    
    df = pd.DataFrame(columns=["path", "label"])  # empty dataframe
    
    for i in CLASS_FOLDERS_PATHS:
        # list files - images
        list_of_files = os.listdir(f"{DATASET_NAME}/{i}")
        
        for k, name in enumerate(list_of_files):
            list_of_files[k] = f"{i}/"+name       # add parent folder

        df = pd.concat([df, pd.DataFrame(list_of_files, columns=["path"])], ignore_index=True)
        df.fillna(labels[i], inplace=True)  # label column is NaN, so we put labels every iteration. because every folder has one class        
    
#     df.reset_index(inplace = True)  
    
    return df

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   path    336 non-null    object
 1   label   336 non-null    object
dtypes: object(2)
memory usage: 5.4+ KB


# CNN Model

In [10]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import StratifiedKFold

In [11]:
width = 480
height = 480

input_size = (width, height)
input_shape = (width, height,3)

batch_size = 16
val_batch_size = 8
test_batch_size = 8

epochs = 10

resnet50 = tf.keras.applications.resnet50.ResNet50(include_top=False,
                                                   weights='imagenet',
                                                   input_shape=input_shape,
                                                   pooling=None)

resnet50.trainable = False


def build_model(input_shape=input_shape):

    inputs = keras.Input(shape=input_shape)
    x = tf.keras.applications.resnet50.preprocess_input(inputs)
    x = resnet50(x)
    x = layers.Flatten()(x)
    
    x = layers.Dense(64, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dense(3, activation="softmax")(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
    
    return model

# K-fold Validation - 5 splits

In [11]:
SUB_FOLDERS_PATH

'../data/20210708_görüntüler'

In [12]:
# stratified k-fold splits data for a percentage of samples for each class
cv = StratifiedKFold(n_splits=5,shuffle=False)

for train_index, test_index in cv.split(df, df.label):
    print("length of train_index: ", len(train_index))
    print("length of test_index: ", len(test_index))
    print()

length of train_index:  268
length of test_index:  68

length of train_index:  269
length of test_index:  67

length of train_index:  269
length of test_index:  67

length of train_index:  269
length of test_index:  67

length of train_index:  269
length of test_index:  67



In [13]:
k = 0

all_train_scores = []   # all train scores on k-fold test data
all_val_scores = []   # all validation scores on k-fold test data


for train_index, test_index in cv.split(df, df.label):
    
    df_train = df.iloc[train_index]
    df_test = df.iloc[test_index]
    
    print("\n-----------------------------")
    print('Initializing Kfold %s'%str(k))
    print('Train shape:', df_train.shape)
    print('Test shape:', df_test.shape)
    

    
    # rescaling was not used because rescaling was used in model layers.    
    train_datagen = ImageDataGenerator()

    train_generator = train_datagen.flow_from_dataframe(dataframe=df_train,
                                                directory=SUB_FOLDERS_PATH,
                                                x_col="path",
                                                y_col="label",
                                                batch_size=batch_size,
                                                seed=42,
                                                shuffle=True,
                                                class_mode="categorical",
                                                target_size=input_size)

    validation_generator = train_datagen.flow_from_dataframe(dataframe=df_test,
                                                directory=SUB_FOLDERS_PATH,
                                                x_col="path",
                                                y_col="label",
                                                batch_size=val_batch_size,
                                                seed=42,
                                                shuffle=True,
                                                class_mode="categorical",
                                                target_size=input_size)
    
    model = build_model(input_shape)
    model.fit(train_generator, epochs=epochs, batch_size=batch_size, verbose=0)
    
    train_generator.reset()
    train_loss, train_accuracy = model.evaluate(train_generator, verbose=0)
    print("train_loss: ", train_loss, "  train_accuracy", train_accuracy)
    all_train_scores.append(train_accuracy)
    
    validation_generator.reset()
    val_loss, val_accuracy = model.evaluate(validation_generator, verbose=0)
    print("val_loss: ", val_loss, "  val_accuracy: ", val_accuracy)
    all_val_scores.append(val_accuracy)
    
    k+=1
    


-----------------------------
Initializing Kfold 0
Train shape: (268, 2)
Test shape: (68, 2)
Found 268 validated image filenames belonging to 3 classes.
Found 68 validated image filenames belonging to 3 classes.
train_loss:  0.027168313041329384   train_accuracy 0.9925373196601868
val_loss:  0.5896450281143188   val_accuracy:  0.8235294222831726

-----------------------------
Initializing Kfold 1
Train shape: (269, 2)
Test shape: (67, 2)
Found 269 validated image filenames belonging to 3 classes.
Found 67 validated image filenames belonging to 3 classes.
train_loss:  0.0011068370658904314   train_accuracy 1.0
val_loss:  0.9932782649993896   val_accuracy:  0.8208954930305481

-----------------------------
Initializing Kfold 2
Train shape: (269, 2)
Test shape: (67, 2)
Found 269 validated image filenames belonging to 3 classes.
Found 67 validated image filenames belonging to 3 classes.
train_loss:  0.0002739644260145724   train_accuracy 1.0
val_loss:  0.6992576718330383   val_accuracy:  

In [14]:
print("All train scores: ", all_train_scores)
print("\nMean of all train scores: ", np.mean(all_train_scores))
print("\nAll validation scores: ", all_val_scores)
print("\nMean of all validation scores: ", np.mean(all_val_scores))

All train scores:  [0.9925373196601868, 1.0, 1.0, 1.0, 1.0]

Mean of all train scores:  0.9985074639320374

All validation scores:  [0.8235294222831726, 0.8208954930305481, 0.8507462739944458, 0.8208954930305481, 0.89552241563797]

Mean of all validation scores:  0.8423178195953369


# K-fold Validation - 10 splits

In [12]:
# stratified k-fold splits data for a percentage of samples for each class
cv = StratifiedKFold(n_splits=10,shuffle=False)

for train_index, test_index in cv.split(df, df.label):
    print("length of train_index: ", len(train_index))
    print("length of test_index: ", len(test_index))
    print()

length of train_index:  302
length of test_index:  34

length of train_index:  302
length of test_index:  34

length of train_index:  302
length of test_index:  34

length of train_index:  302
length of test_index:  34

length of train_index:  302
length of test_index:  34

length of train_index:  302
length of test_index:  34

length of train_index:  303
length of test_index:  33

length of train_index:  303
length of test_index:  33

length of train_index:  303
length of test_index:  33

length of train_index:  303
length of test_index:  33



In [13]:
%%time

k = 0

all_train_scores = []   # all train scores on k-fold test data
all_val_scores = []   # all validation scores on k-fold test data


for train_index, test_index in cv.split(df, df.label):
    
    df_train = df.iloc[train_index]
    df_test = df.iloc[test_index]
    
    print("\n-----------------------------")
    print('Initializing Kfold %s'%str(k))
    print('Train shape:', df_train.shape)
    print('Test shape:', df_test.shape)
    

    
    # rescaling was not used because rescaling was used in model layers.    
    train_datagen = ImageDataGenerator()

    train_generator = train_datagen.flow_from_dataframe(dataframe=df_train,
                                                directory=SUB_FOLDERS_PATH,
                                                x_col="path",
                                                y_col="label",
                                                batch_size=batch_size,
                                                seed=42,
                                                shuffle=True,
                                                class_mode="categorical",
                                                target_size=input_size)

    validation_generator = train_datagen.flow_from_dataframe(dataframe=df_test,
                                                directory=SUB_FOLDERS_PATH,
                                                x_col="path",
                                                y_col="label",
                                                batch_size=val_batch_size,
                                                seed=42,
                                                shuffle=True,
                                                class_mode="categorical",
                                                target_size=input_size)
    
    model = build_model(input_shape)
    model.fit(train_generator, epochs=epochs, batch_size=batch_size, verbose = 0)
    
    train_generator.reset()
    train_loss, train_accuracy = model.evaluate(train_generator, verbose=0)
    print("train_loss: ", train_loss, "  train_accuracy", train_accuracy)
    all_train_scores.append(train_accuracy)
    
    validation_generator.reset()
    val_loss, val_accuracy = model.evaluate(validation_generator, verbose=0)
    print("val_loss: ", val_loss, "  val_accuracy: ", val_accuracy)
    all_val_scores.append(val_accuracy)
    
    k+=1
    


-----------------------------
Initializing Kfold 0
Train shape: (302, 2)
Test shape: (34, 2)
Found 302 validated image filenames belonging to 3 classes.
Found 34 validated image filenames belonging to 3 classes.


ResourceExhaustedError: Graph execution error:

Detected at node 'model/resnet50/conv2_block1_3_bn/FusedBatchNormV3' defined at (most recent call last):
    File "C:\Users\ekizo\anaconda3\envs\tensorflow\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\ekizo\anaconda3\envs\tensorflow\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\ekizo\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\ekizo\anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\ekizo\anaconda3\envs\tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\ekizo\anaconda3\envs\tensorflow\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run

In [14]:
print("All train scores: ", all_train_scores)
print("\nMean of all train scores: ", np.mean(all_train_scores))
print("\nAll validation scores: ", all_val_scores)
print("\nMean of all validation scores: ", np.mean(all_val_scores))

All train scores:  []

Mean of all train scores:  nan

All validation scores:  []

Mean of all validation scores:  nan


C:\Users\ekizo\anaconda3\envs\tensorflow\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\ekizo\anaconda3\envs\tensorflow\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
